<a href="https://www.kaggle.com/code/rahulchauhan016/rc-fastinstructionlm-cpu-gguf-model?scriptVersionId=294749640" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 1 – Install Packages

In [1]:
!pip install -q bitsandbytes peft accelerate transformers datasets gradio




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.22.1 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
langchain-core 0.3.79 requires packaging<26.0.0,>=23.2.0, but you have packaging 26.0rc2 which is incompatible.
fastai 2.8.4 requires fastcore<1.9,>=1.8.0, but you have fastcore 1.11.3 which is incompatible.


# 2 - Imports

In [2]:

import os
import shutil
import torch
from datasets import load_from_disk
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, PeftModel


2026-01-29 11:43:34.689007: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769687014.907670      25 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769687014.970449      25 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769687015.513089      25 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769687015.513140      25 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769687015.513143      25 computation_placer.cc:177] computation placer alr

# 3 - Load Conversation Dataset

In [3]:

SRC_TRAIN = "/kaggle/input/lexor-2026/150k_conversations_14M_tokens/final_train_dataset"
SRC_VAL   = "/kaggle/input/lexor-2026/150k_conversations_14M_tokens/final_val_dataset"

DST_TRAIN = "/kaggle/working/train_dataset"
DST_VAL   = "/kaggle/working/val_dataset"

if not os.path.exists(DST_TRAIN):
    shutil.copytree(SRC_TRAIN, DST_TRAIN)

if not os.path.exists(DST_VAL):
    shutil.copytree(SRC_VAL, DST_VAL)

train_dataset = load_from_disk(DST_TRAIN)
val_dataset   = load_from_disk(DST_VAL)

print("Train samples:", len(train_dataset))
print("Val samples:", len(val_dataset))


Train samples: 158538
Val samples: 17616


# 4 – Ensure labels exist

In [4]:

def ensure_labels(ds):
    if "labels" not in ds.column_names:
        ds = ds.map(lambda x: {"labels": x["input_ids"]})
    return ds

train_dataset = ensure_labels(train_dataset)
val_dataset   = ensure_labels(val_dataset)

print("Columns:", train_dataset.column_names)


Map:   0%|          | 0/158538 [00:00<?, ? examples/s]

Map:   0%|          | 0/17616 [00:00<?, ? examples/s]

Columns: ['input_ids', 'attention_mask', 'labels']


# 5 – Load base model in 4-bit (QLoRA)

In [5]:
import bitsandbytes as bnb
print("bitsandbytes version:", bnb.__version__)


import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL_NAME = "EleutherAI/pythia-1.4b"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)

print("✅ Model loaded in 4-bit with BitsAndBytesConfig")


bitsandbytes version: 0.49.1


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.93G [00:00<?, ?B/s]

✅ Model loaded in 4-bit with BitsAndBytesConfig


# 6 – Attach LoRA adapters

In [6]:

from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value", "dense"],  # correct for Pythia
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()


trainable params: 4,718,592 || all params: 1,419,366,400 || trainable%: 0.3324


# 6.2 - Select training subset (20k best balance)


In [7]:


TRAIN_SAMPLES = 20000
VAL_SAMPLES = 3000

train_dataset_fast = train_dataset.shuffle(seed=42).select(range(TRAIN_SAMPLES))
val_dataset_fast   = val_dataset.shuffle(seed=42).select(range(VAL_SAMPLES))

print("Fast train size:", len(train_dataset_fast))
print("Fast val size:", len(val_dataset_fast))


Fast train size: 20000
Fast val size: 3000


# 7 – Training configuration (QLoRA)

In [8]:

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./qlora_fast_ckpt",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=2,        # 👈 BEST choice
    fp16=True,
    save_strategy="no",
    logging_steps=20,
    report_to="none",
    optim="paged_adamw_8bit"
)


# 8 – Train

In [9]:


from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_fast,
    eval_dataset=val_dataset_fast
)

trainer.train()


Step,Training Loss
20,4.832900
40,2.664800
60,2.325500
80,2.218400
100,2.197500
120,2.035100
140,2.193900
160,2.273900
180,2.185500
200,2.038300


TrainOutput(global_step=2500, training_loss=1.8779644523620604, metrics={'train_runtime': 9519.1521, 'train_samples_per_second': 4.202, 'train_steps_per_second': 0.263, 'total_flos': 8.087616356352e+16, 'train_loss': 1.8779644523620604, 'epoch': 2.0})

# 9 – Save trained model

In [10]:


SAVE_PATH = "/kaggle/working/fast_model"

model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)

print("Model saved to:", SAVE_PATH)


Model saved to: /kaggle/working/fast_model


In [11]:
# %%
# Cell B - Verify save folder

import os

print("Contents of /kaggle/working:")
print(os.listdir("/kaggle/working"))

print("\nContents of model folder:")
print(os.listdir("/kaggle/working/fast_model"))



Contents of /kaggle/working:
['val_dataset', 'qlora_fast_ckpt', '__notebook__.ipynb', 'train_dataset', 'fast_model']

Contents of model folder:
['tokenizer_config.json', 'adapter_model.safetensors', 'tokenizer.json', 'README.md', 'adapter_config.json', 'special_tokens_map.json']


# 10 –  Load base model

In [12]:


from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

BASE_MODEL = "EleutherAI/pythia-1.4b"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    torch_dtype=torch.float16
)

print("Base model loaded")



`torch_dtype` is deprecated! Use `dtype` instead!


Base model loaded


# 10.1 -  Load LoRA adapters

In [13]:

from peft import PeftModel

LORA_PATH = "/kaggle/working/fast_model"

model = PeftModel.from_pretrained(base_model, LORA_PATH)

print("LoRA adapters loaded")


LoRA adapters loaded


# 10.2 - Merge LoRA into full model

In [14]:


MERGED_PATH = "/kaggle/working/fast_model_merged"

merged_model = model.merge_and_unload()
merged_model.save_pretrained(MERGED_PATH)
tokenizer.save_pretrained(MERGED_PATH)

print("✅ Full model saved to:", MERGED_PATH)


✅ Full model saved to: /kaggle/working/fast_model_merged


# 10.3  - Load merged model only

In [15]:

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

MODEL_PATH = "/kaggle/working/fast_model_merged"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    torch_dtype=torch.float16
)

print("✅ Merged model loaded")


✅ Merged model loaded


# 11 – Test chat

In [16]:

def test_model(prompt):
    text = f"Instruction: {prompt}\nAnswer:"
    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    out = model.generate(**inputs, max_new_tokens=120)
    print(tokenizer.decode(out[0], skip_special_tokens=True))

test_model("Give 3 benefits of artificial intelligence.")


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Instruction: Give 3 benefits of artificial intelligence.
Answer: Give 3 benefits of artificial intelligence.
Answer: Give 3 benefits of artificial intelligence.
Answer: Give 3 benefits of artificial intelligence.
Answer: Give 3 benefits of artificial intelligence.
Answer: Give 3 benefits of artificial intelligence.
Answer: Give 3 benefits of artificial intelligence.
Answer: Give 3 benefits of artificial intelligence.
Answer: Give 3 benefits of artificial intelligence.
Answer: Give 3 benefits of artificial intelligence.
Answer: Give 3 benefits of artificial intelligence.
Answer: Give 3 benefits of artificial intelligence.
Answer: Give 3 benefits of artificial intelligence.
Answer:
